# Example of HTML capabilities of project inside IPython / Jupyter notebook

Note, this file used in tests, so it may contain additional test related logic.

## Initializing Environment

In [ ]:
# import extensions first (they modify Session and Client classes)
from openerp_proxy.ext.all import HField

# Enable module_utils plugin
import openerp_proxy.plugins.module_utils

# Import Client and Session classes
from openerp_proxy import (Client,
                           Session)

## Connect to odoo database

For connection to Odoo ```Client``` class is used. Below is example of it's usage.

In [ ]:
cl = Client('localhost')  # connect to local instance of server
cl

Next we should check if database we would like to connect to is present on server.
For this purpose we should use Odoo's database service, which could be accessed via ```Client.services.db```.
And if database does not exists, we can create it.

In [ ]:
# check if our demo database exists
if 'openerp_proxy_test_db' not in cl.services.db:
    # create demo database
    cl.services.db.create_db('admin', 'openerp_proxy_test_db', demo=True, lang='en_US')

And now we can login to our database.

In [ ]:
# login to created database
ldb = cl.login('openerp_proxy_test_db', 'admin', 'admin')  # all this arguments could be passed directly to Client constructor.

# and let's look how it is displayed in IPython
ldb

*Note*, that ```ldb``` is new instance of ```Client``` class, but with login credential. it can be used to interact with object service (models, documents, logic, ...)

## Session class

If You often need to connect to same databases, there are a ```openerp_proxy.Session``` class,
which automaticaly save, most of your connections, made via ```Session.connect``` method in specified file.

In [ ]:
# create session instance
session = Session('~/.openerp_proxy.local.json')  # default file path is '~/.openerp_proxy.json'

And there are option You may be interested in. It is 'store_passwords', which automaticaly saves password You have used for connection.

In [ ]:
session.option('store_passwords', True)

We used ```openerp_proxy.Client``` class to create connection to database, so our session does not know anything about it. Let's add our connection to our session:

In [ ]:
session.add_db(ldb)
session

And now we can get this connection from session by index, or by URL (look at the table above). But to simplify next connections, we may add aliase to this connection

In [ ]:
session.aliase('ldb', ldb)
session

So, now, to get connection again we could just type ```session.ldb```.

Ok. initialization is done, and now we could save it.

In [ ]:
session.save()

## Connect to odoo database  (via session)

Let's now create new instance of session, and connect to created above database

In [ ]:
session = Session('~/.openerp_proxy.local.json')  # default file path is '~/.openerp_proxy.json'
ldb = session.ldb
ldb

## Module Utils plugin

Our database is clean, for next code, we need to install ```sale``` addon. For this, we have ```module_utils``` plugin, out-of-the box, which simplyfies work with modules. To enable this plugin, we just need to import it, and then we will have it in ```ldb.plugins``` property. This plugin was imported above.
This plugin extends ```ir.module.module``` model from client side, adding simple methods: ```install``` and ```upgrade``` to it for shorter syntax. If You're interested for code, look [here](https://github.com/katyukha/openerp-proxy/blob/master/openerp_proxy/plugins/module_utils.py), it is very simple!

So now, let's install ```sale``` module.  (Note that in most cases, **IPython autocompletition** work's fine)

In [ ]:
ldb.plugins.module_utils.m_sale.install()

Congratulation! module was installed! Now we need only to refresh caches, to see new models in database.

In [ ]:
ldb.clean_caches()

## Get list of all registered objects (models)

To get list of registered models, just use *registered_objects* proerty of *Client* instance.
It returns list of all registered models in database. For example:

In [ ]:
ldb.registered_objects

## Get object / model

*ldb* here represents database connection (*Client* class instance)
As told in help message above *.get_obj* method allows to get instance of specified
*Object* proxy, where Object means *model*, *document*.

In [ ]:
so = ldb.get_obj('sale.order')
so

Also it is posible to use shorter (dictionary style) syntax:

In [ ]:
so = ldb['sale.order']
so

And as result of using [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension
(which is automaticaly imported in 'openerp_proxy.ext.all')
there are attribute-style access (which also support's IPython auto-completition):

In [ ]:
so = ldb._sale_order
so

## Getting information about available columns for Object

In [ ]:
so.columns_info

## Search for sale orders

In [ ]:
# Standard search .search_records(domain)
so_list = so.search_records([])
so_list

Also there are shorter syntax provided by [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension:

In [ ]:
so_list = so([])
so_list

## Display Sale orders as HTML table

It is posible to represent RecordList as HTML table, with ability to highlight rows by specified conditions.
Also, when building result table, it is posible to display values of related fields, and even method calls. This functionality is implemented in [*openerp_proxy.ext.repr*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.repr) module

In [ ]:
# High light rows by condition
highlighters = {
    '#99FF99': lambda x: x.state == 'done',
    '#9999FF': lambda x: x.state == 'draft',
    '#FFFF99': lambda x: x.state == 'progress',
}

# Display as table.
# Note that prefetch method is used to fetch some set of fields with less RPC call.
# on big datasets it may speed up performance signifiantly.
# Each RecordList instance have related cache, which reduce need of reading data on each field get.
so_list.prefetch('id', 'name', 'partner_id', 'partner_id.email', 'state')
so_table = so_list.as_html_table(
    'id',
    'name',
    # _name attribute provides result of *name_search method:
    HField('partner_id._name', name='Partner name'),
    # silent=True means, if field cannot be found, not throw error
    HField('partner_id.email', name='Partner email', silent=True),
    # Also it is posible to display result of method calls
    # 'as_html_list()' is method of RecordList.
    ('order_line.as_html_list', 'Order lines'),
    'state',
    highlighters=highlighters,
)
so_table

There also available to_csv method, which allow to represent table in csv format

In [ ]:
so_table.to_csv()

## Access one element of recordlist via index

In [ ]:
so_list[0]


## Display one sale order as HTML Table

In [ ]:
so_list[0].as_html('name',
                   'origin',
                   'partner_id',        # Will display Reacord instance representing partner related to this sale order
                   'partner_id._name',  # Will display result of 'name_get' called on partner
                   'partner_id.sale_order_ids.length')  # Display how many sale orders have this partner

In [ ]:
so_list[0].as_html()  # Display all fields for firest sale order record

## Report service

There is ``reports`` service available in Odoo, which allows to print reports.
Below example usage of it.

Access the service:

In [ ]:
ldb.services.report

Get list of available reports:

In [ ]:
ldb.services.report.available_reports

Choose report name for current Odoo server version

In [ ]:
from pkg_resources import parse_version as V
if ldb.server_version >= V("8.0"):
    report_name = 'sale.report_saleorder'
else:
    report_name = 'sale.order'

report_name

Take a look on report object. It also has it's own HTML representation.

In [ ]:
report = ldb.services.report[report_name]
report

And generate new report:

In [ ]:
report_result = report.generate(so_list)
report_result